In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatOpenAI()


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [ ]:
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

# instead of worklow saying chatbot as chatbot is also a workflow but simple 
chatbot = graph.compile()

In [ ]:
chatbot   

In [ ]:
# doesnot allow for continuous chatting.
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

# if chatbot.invoke(initial_state)  gives both human and ai message appenended 
chatbot.invoke(initial_state)['messages'][-1].content  

In [ ]:
# allow for continuous chatting.
while True:

user_message = input('Type here: ')
print('User:', user_message)

if user_message.strip().lower() in ['exit', 'quit', 'bye']:

break

response = chatbot. invoke({'messages': [HumanMessage(content=user_message)]})

print('AI:', response['messages'][-1].content)